In [6]:
import sys
sys.path.insert(0, '../code')
import dismod_input
from dismod_db import DismodDB
from two_level import TwoLevel
from plot_two_level import PlotTwoLevel
import numpy as np
import pandas as pd
from dismod_output import DismodOutput

ModuleNotFoundError: No module named 'dismod_at'

In [2]:
data = dismod_input.get_input_from_db(location_ids=[71, 72])

2019-11-22 10:01:08,008 --- INFO ---  elmo.run.run_request ---  preparing to execute request --- [request_id:397214]
2019-11-22 10:01:08,021 --- INFO ---  elmo.run.run_request ---  beginning data import --- [request_id:397214]
2019-11-22 10:01:12,691 --- INFO ---  elmo.run.run_request ---  beginning data transformation --- [request_id:397214]
2019-11-22 10:01:15,829 --- INFO ---  elmo.run.run_request ---  beginning create tables --- [request_id:397214]
2019-11-22 10:01:15,832 --- INFO ---  elmo.run.run_request ---  beginning data write --- [request_id:397214]
2019-11-22 10:01:15,834 --- INFO ---  elmo.run.run_request ---  Committing changes --- [request_id:397214]
2019-11-22 10:01:15,836 --- INFO ---  elmo.run.run_request ---  done committing --- [request_id:397214]
2019-11-22 10:01:15,840 --- INFO ---  elmo.run.run_request ---  process finished --- [request_id:397214]


In [4]:
data.head()

,location_name,location_id,sex,year_start,year_end,age_start,age_end,measure,mean,standard_error
0,Taiwan (Province of China),8,Female,1999,2000,0.0,15.0,incidence,0.000044,0.000004
1,Taiwan (Province of China),8,Female,2001,2002,0.0,15.0,incidence,0.000049,0.000004
2,Taiwan (Province of China),8,Female,2003,2004,0.0,15.0,incidence,0.000058,0.000005
3,Taiwan (Province of China),8,Female,2005,2006,0.0,15.0,incidence,0.000055,0.000005
4,Taiwan (Province of China),8,Female,2007,2008,0.0,15.0,incidence,0.000062,0.000006


In [5]:
rates = ['iota']
meas_noise_density = {'Sincidence': {'density': 'gaussian'}}
rates_parent_priors = [
    # rate priors for iota
    ({'density':'uniform','upper':1.0,'lower':1e-16,'mean': .0001},\
    {'density': 'gaussian', 'mean':0.0, 'std': .001},\
    {'density': 'gaussian', 'mean':0.0, 'std': .001})]
rates_child_priors = [
    # rate priors for iota
    ({'density':'gaussian','mean': 0.0, 'std': .1}, None, None)]
#---------change path------------
path_to_db = './data/incidence_two_level_loop.db'

locations = ['Australia', 'New Zealand']
integrand = ['Sincidence']
db = DismodDB(data, locations, integrand,rates, rates_parent_priors, 
              rates_child_priors, meas_noise_density,path_to_db)

NameError: name 'DismodDB' is not defined

In [ ]:
model = TwoLevel(db)
model.fit_both(zero_sum=True, tol=1e-8)
path = './data/'
plt = PlotTwoLevel(path, 'incidence_two_level.db')
plt.plot_residuals('all')
plt.plot_change_over_age('rate', 'iota', 'Sincidence', 'Australia')

In [ ]:
output = DismodOutput(path_to_db)
model_version_id = 395838
output.save_GBD_output(['Sincidence', 'prevalence'], model_version_id, list(range(1980, 2019)), 
                       [1, 2], location_name_to_id, './data/incidence_gbd_sample.csv')
